Datasets Used:
<ul>
    <li>https://data.wprdc.org/dataset/fire-incidents-in-city-of-pittsburgh Weight:</li>
    <li>https://data.wprdc.org/dataset/police-incident-blotter Weight: </li>
    <li>https://data.wprdc.org/dataset/pbp-fire-arm-seizures Weight: </li>
    <li>https://data.wprdc.org/dataset/arrest-data Weight: </li>
    <li>https://data.wprdc.org/dataset/landslide-prone-areas geojson</li>
    <li>https://data.wprdc.org/dataset/allegheny-county-fatal-accidental-overdoses sorted by zip</li>
    <li>https://data.wprdc.org/dataset/hospitals</li>
</ul>
Intro: <br>
Our 'best' neighborhood is being calculated by the safety of the neighborhood. This is determined by data such as fire incidents, arrests, fire arm seizures, and police incidents. Each neighborhood is given a percent score based on the number of occurences in that neighborhood, and then the overall percent is multiplied by a weight based on the importance of that dataset. There are also negative scores for data such as hospitals. The 'best' neighborhood will have the lowest score.

In [1]:
import pandas as pd

In [7]:
neighborhoods = pd.read_csv("Neighborhoods.csv")
hoodsdict = neighborhoods['hood'].value_counts().to_dict()
neighborhoods.set_index('hood')
print(neighborhoods[neighborhoods.hood == "West End"]['sqmiles'])

fire = pd.read_csv("fire-incidents.csv")
firedict = fire['neighborhood'].value_counts().to_dict()

arrest = pd.read_csv("arrest-data.csv")
arrestdict = arrest['INCIDENTNEIGHBORHOOD'].value_counts().to_dict()

policeblotter = pd.read_csv("police-blotter.csv")
policedict = policeblotter['INCIDENTNEIGHBORHOOD'].value_counts().to_dict()

scores = hoodsdict

for key in firedict:
    if key in scores:
        scores[key] = scores[key] + firedict[key]
    else:
        scores[key] = firedict[key]

for key in arrestdict:
    if key in scores:
        scores[key] = scores[key] + arrestdict[key]
    else:
        scores[key] = arrestdict[key]
        
for key in policedict:
    if key in scores:
        scores[key] = scores[key] + policedict[key]
    else:
        scores[key] = policedict[key]
        
#cleaning the dictionary, combining duplicates and getting rid of non-pittsburgh neighborhoods
scores['Mt. Oliver'] = scores['Mt. Oliver Neighborhood'] + scores['Mt. Oliver Neighborhood'] + scores['Mount Oliver']
scores['Central Business District'] = scores['Central Business District'] + scores['Golden Triangle/Civic Arena']
scores['Central Northside'] = scores['Central Northside'] + scores['Central North Side']
scores['Troy Hill'] = scores['Troy Hill'] + scores['Troy Hill-Herrs Island']
del scores['Troy Hill-Herrs Island']
del scores['Central North Side']
del scores['Golden Triangle/Civic Arena']
del scores['Mt. Oliver Neighborhood']
del scores['Mount Oliver']
del scores['Mt. Oliver Boro']
del scores['Mount Oliver Borough']
del scores['Outside County']
del scores['Outside State']
del scores['Outside City']

#for key in scores:
    #scores[key] = scores[key] / neighborhoods[neighborhoods.hood == key]['sqmiles']

s = pd.Series(scores, name="Score")
s.index.name = "Neighborhood"
s.sort_values().head()

86    0.2
Name: sqmiles, dtype: float64


Neighborhood
Regent Square     28
Ridgemont         29
Chartiers City    35
Swisshelm Park    35
East Carnegie     36
Name: Score, dtype: int64